In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import gaussian_kde
from scipy.stats import norm

In [ ]:
def get_model_rewards(model_name, group):
    model_pth = f'../results/rewards/{group}/sub-'+model_name+'/'+'rewards_30.npy'
    model_rewards = np.load(model_pth)
    return model_rewards

In [ ]:
def get_subject_reward_paths(model_name):
    subject_rewards_1_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d1.csv'
    subject_rewards_2_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d2.csv'
    subject_rewards_3_pth = '/Users/hazimiasad/Documents/Work/megan/data/collection/Study1/sub-'+model_name+'/pattern/in_score_d3.csv'
    all_subject_reward_pths = [subject_rewards_1_pth, subject_rewards_2_pth, subject_rewards_3_pth]
    return all_subject_reward_pths

In [ ]:
def get_subject_rewards(all_subject_reward_pths, limit):
    subject_rewards = [pd.read_csv(pth, header=None).values[:limit] for pth in all_subject_reward_pths]
    subject_rewards = np.vstack(subject_rewards)
    return subject_rewards

In [ ]:
def fit_gaussian_and_compute_nll(train_data, test_data):
    # Fit a Gaussian distribution to the training data
    mu, sigma = np.mean(train_data), np.std(train_data, ddof=1)  # Unbiased estimator
    
    # Compute the negative log-likelihood (NLL) for the test data
    nll = -np.sum(norm.logpdf(test_data, loc=mu, scale=sigma))
    
    return mu, sigma, nll

In [ ]:
def calc_nlls(model_rewards, subject_rewards):
    nlls = np.zeros((len(model_rewards), np.shape(model_rewards)[2]))
    for idx, model_reward in enumerate(model_rewards):
        for rep in range(np.shape(model_rewards)[2]):
            _, _, nll = fit_gaussian_and_compute_nll(
                subject_rewards, model_reward[:, rep]
            )
            nlls[idx, rep] = nll
    return nlls

In [ ]:
def calculate_min_nlls_and_indices(group):
    min_nlls = np.zeros(24)
    min_nll_indices = np.zeros(24)
    for i in range(1, 25):
        model_name = f"{i:02}"
        model_rewards = get_model_rewards(model_name, group)
        all_subject_reward_pths = get_subject_reward_paths(model_name)
        subject_rewards = get_subject_rewards(all_subject_reward_pths, 15)
        nlls = calc_nlls(model_rewards, subject_rewards)
        nlls_mean = np.mean(nlls, axis=0)
        min_nll = np.min(nlls_mean)
        min_nll_index = np.argmin(nlls_mean)
        min_nlls[i-1] = min_nll
        min_nll_indices[i-1] = min_nll_index
    return min_nlls, min_nll_indices

In [ ]:
min_nlls_rl, min_nll_indices_rl = calculate_min_nlls_and_indices('rl')
min_nlls_gradient, min_nll_indices_gradient = calculate_min_nlls_and_indices('gradient')


In [ ]:
bins = 20
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Histogram for min_nlls_rl and min_nlls_gradient
axs[0].hist(min_nlls_rl, bins=bins, alpha=1, label='RL-Diffusion', color='blue')
axs[0].hist(min_nlls_gradient, bins=bins, alpha=0.5, label='Control-Diffusion', color='red')
axs[0].set_title('Histogram of NLL Values')
axs[0].set_xlabel('NLL Values')
axs[0].set_ylabel('Frequency')
axs[0].legend()
axs[0].set_yticks([axs[0].get_yticks()[0], axs[0].get_yticks()[-1]])
axs[0].set_xticks([0,  axs[0].get_xticks()[int(len(axs[0].get_xticks())/2)] , axs[0].get_xticks()[-2]])

# Histogram for min_nll_indices_rl and min_nll_indices_gradient
axs[1].hist(min_nll_indices_rl*10, bins=bins, alpha=1, label='RL-Diffusion', color='blue')
axs[1].hist(min_nll_indices_gradient*3, bins=bins, alpha=0.5, label='Control-Diffusion', color='red')
axs[1].set_title('Histogram of min indices')
axs[1].set_xlabel('Training Epochs')
axs[1].set_ylabel('Frequency')
axs[1].set_xticks([1, 150, 300])
axs[1].legend()
axs[1].set_yticks([axs[1].get_yticks()[0], axs[1].get_yticks()[-1]])

fig.suptitle('NLL Analysis')
plt.tight_layout()
plt.show()


In [ ]:
def get_all_model_rewards(group):
    all_model_rewards = []
    for i in range(1, 25):
        model_name = f"{i:02}"
        model_rewards = get_model_rewards(model_name, group)
        all_model_rewards.append(model_rewards)
    return all_model_rewards

all_model_rewards_rl = get_all_model_rewards('rl')
all_model_rewards_gradient = get_all_model_rewards('gradient')

In [ ]:
all_model_rewards_rl = [np.mean(rewards, axis=2) for rewards in all_model_rewards_rl]
all_model_rewards_gradient = [np.mean(rewards, axis=2) for rewards in all_model_rewards_gradient]

In [ ]:
all_model_rewards_rl = [np.mean(rewards, axis=1) for rewards in all_model_rewards_rl]
all_model_rewards_gradient = [np.mean(rewards, axis=1) for rewards in all_model_rewards_gradient]

In [ ]:
for i, ii in enumerate(all_model_rewards_rl):
    print(ii.shape, min_nll_indices_rl[i])

In [ ]:
# Extract the y values based on min_nll_indices_rl
y_values_rl = []
for i in range(len(min_nll_indices_rl)):
    index = int(min_nll_indices_rl[i])
    if index < len(all_model_rewards_rl[i]):
        y_values_rl.append(all_model_rewards_rl[i][index])
    else:
        y_values_rl.append(np.nan)  # Handle out-of-bounds index

# Extract the y values based on min_nll_indices_gradient
y_values_gradient = []
for i in range(len(min_nll_indices_gradient)):
    index = int(min_nll_indices_gradient[i])
    if index < len(all_model_rewards_gradient[i]):
        y_values_gradient.append(all_model_rewards_gradient[i][index])
    else:
        y_values_gradient.append(np.nan)  # Handle out-of-bounds index

# Create the scatter plot for both RL and Gradient
plt.scatter(min_nlls_rl, y_values_rl, label='RL-Diffusion', color='blue')
plt.scatter(min_nlls_gradient, y_values_gradient, label='Control-Diffusion', color='red')
plt.xlabel('Min NLL Value')
plt.ylabel('All Model Rewards [Min NLL Index]')
plt.title('Scatter Plot of Min NLL Values vs All Model Rewards')
plt.legend()
plt.show()
